In [2]:
import pandas as pd
import glob
import os
import numpy as np
from pathlib import Path
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import torch

In [ ]:
INPUT_DIR = "../data/news/processed/marketaux"
OUTPUT_DIR = "../data/news/processed/marketaux_roberta"
MODEL_NAME = "cardiffnlp/twitter-xlm-roberta-base-sentiment"

Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

print(f"🔄 Carregando modelo RoBERTa ({MODEL_NAME})...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
print(f"🚀 Rodando inferência em: {device.upper()}")

🔄 Carregando modelo RoBERTa (cardiffnlp/twitter-xlm-roberta-base-sentiment)...


🚀 Rodando inferência em: CPU


In [ ]:

def calcular_roberta_score(texto):
    """
    Retorna um score único entre -1 e 1.
    Lógica: (Prob_Positiva - Prob_Negativa)
    """
    if not isinstance(texto, str) or len(texto) < 3:
        return 0.0

    try:
        encoded_input = tokenizer(texto, return_tensors='pt', truncation=True, max_length=512).to(device)
        
        with torch.no_grad():
            output = model(**encoded_input)
        
        scores = output.logits[0].cpu().numpy()
        probs = softmax(scores)
        
        neg = probs[0]
        neu = probs[1]
        pos = probs[2]
        
        compound_score = pos - neg
        
        return compound_score
        
    except Exception as e:
        # print(f"Erro no texto: {texto[:20]}... {e}")
        return 0.0


In [ ]:
def main_processamento():
    arquivos = glob.glob(os.path.join(INPUT_DIR, "*.parquet"))
    print(f"📂 Encontrados {len(arquivos)} arquivos para processar.")

    for arquivo in tqdm(arquivos, desc="RoBERTa em ação"):
        try:
            df = pd.read_parquet(arquivo)
            if df.empty: continue

            df['texto_full'] = df['title'].fillna('') + ". " + df['description'].fillna('')
            
            tqdm.pandas(desc=f"Lendo {os.path.basename(arquivo)}")
            df['sentiment'] = df['texto_full'].progress_apply(calcular_roberta_score)
            
            df = df.drop(columns=['texto_full'], errors='ignore')
            
            nome_saida = os.path.basename(arquivo)
            df.to_parquet(os.path.join(OUTPUT_DIR, nome_saida), index=False)
            
        except Exception as e:
            print(f"❌ Falha em {arquivo}: {e}")

    print(f"\n✅ Concluído! Dados padronizados em: {OUTPUT_DIR}")

if __name__ == "__main__":
    main_processamento()

📂 Encontrados 97 arquivos para processar.


RoBERTa em ação: 100%|██████████| 97/97 [00:42<00:00,  2.31it/s]


✅ Concluído! Dados padronizados em: ../data/news/processed/marketaux_roberta
